In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost


     |████████████████████████████████| 76.4 MB 27 kB/s 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold,train_test_split,KFold
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from catboost import Pool,CatBoostClassifier
import warnings

In [ ]:
def freq_enc(df,cols):
    for col in cols:
        df[col] = df[col].map(df[col].value_counts().to_dict())/len(df)
    return df

In [ ]:
train_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PredictH1N1/training_set_features.csv')
train_label = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PredictH1N1/training_set_labels.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PredictH1N1/test_set_features.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PredictH1N1/submission_format.csv')

In [ ]:
train_set.shape

(26707, 36)

In [ ]:
train_set.isnull().sum()

h1n1_concern                      92
h1n1_knowledge                   116
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_h1n1                2160
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_h1n1_vacc_effective      391
opinion_h1n1_risk                388
opinion_h1n1_sick_from_vacc      395
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
r

In [ ]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

In [ ]:
test_copy = test_set.copy()
train_copy = train_set.copy()

In [ ]:
train_set = train_set.set_index('respondent_id')
test_set = test_set.set_index('respondent_id')

KeyError: ignored

In [ ]:
y1 = train_label['h1n1_vaccine'].values
y2 = train_label['seasonal_vaccine'].values
train_label.head()


In [ ]:
numeric_col = np.where(train_set.dtypes != object)[0]
numeric_feat = [train_set.columns[x] for x in numeric_col]


In [ ]:
train_cluster_df = train_set[numeric_feat]
test_cluster_df =  test_set[numeric_feat]

train_cluster_df = train_cluster_df.fillna(-999)
test_cluster_df = test_cluster_df.fillna(-999)

In [ ]:
kmeans = KMeans(4,random_state=42)
kmeans.fit(train_cluster_df)
train_set['clusters'] = kmeans.predict(train_cluster_df)
test_set['clusters']  = kmeans.predict(test_cluster_df)

In [ ]:
train_cluster_df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children,clusters
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,1
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,0.0,0.0,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-999.0,-999.0,1.0,0.0,0.0,-999.0,3.0,1.0,1.0,4.0,1.0,2.0,2.0,0.0,2
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-999.0,3.0,3.0,5.0,5.0,4.0,1.0,0.0,0.0,0
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-999.0,3.0,3.0,2.0,3.0,1.0,4.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-999.0,3.0,1.0,1.0,5.0,2.0,2.0,0.0,0.0,0
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,1.0,0.0,1
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-999.0,4.0,4.0,2.0,5.0,4.0,2.0,0.0,0.0,0


In [ ]:
train_set.columns

Index(['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation', 'clusters'],
      dtype='object')

In [ ]:
cols = ['behavioral_face_mask','behavioral_wash_hands','behavioral_touch_face',
        'opinion_seas_vacc_effective','opinion_seas_risk'
        ,'opinion_h1n1_vacc_effective','opinion_h1n1_risk'
    ,'opinion_seas_vacc_effective' ,'opinion_seas_sick_from_vacc'    
     ,'opinion_h1n1_vacc_effective' , 'opinion_h1n1_sick_from_vacc'
     ,'h1n1_concern', 'h1n1_knowledge'
     ,'household_adults','household_children'
     ,'behavioral_large_gatherings' , 'behavioral_outside_home']


In [ ]:
for col in cols:
    train_set[col] = train_set[col].apply(str)
    test_set[col] = test_set[col].apply(str)
    

In [ ]:
train_set['total_persons'] = train_set['household_adults']+train_set['household_children']
test_set['total_persons'] = test_set['household_adults']+test_set['household_children']

In [ ]:

train_set['face_mask_wash_hand'] = train_set['behavioral_face_mask']+train_set['behavioral_wash_hands']
train_set['face_mask_touch_face'] = train_set['behavioral_face_mask'] + train_set['behavioral_touch_face']
train_set['wash_hands_touch_face'] = train_set['behavioral_wash_hands']+train_set['behavioral_touch_face']
train_set['opinion_seas_eff_risk'] = train_set['opinion_seas_vacc_effective']+train_set['opinion_seas_risk']
train_set['opinion_h1n1_eff_risk'] = train_set['opinion_h1n1_vacc_effective']+train_set['opinion_h1n1_risk']
train_set['opinio_seas_vacc_eff_sick'] = train_set['opinion_seas_vacc_effective'] + train_set['opinion_seas_sick_from_vacc']
train_set['opinon_h1n1_vacc_eff_sick'] = train_set['opinion_h1n1_vacc_effective']+train_set['opinion_h1n1_sick_from_vacc']
train_set['h1n1_concern_knowledge']   = train_set['h1n1_concern'] + train_set['h1n1_knowledge']
train_set['behaviour_large_gath_outside_home']  = train_set['behavioral_large_gatherings'] + train_set['behavioral_outside_home']
train_set['opinion_seas_vaccine']  = train_set['opinion_seas_risk']+train_set['opinion_seas_sick_from_vacc']
train_set['opinion_h1n1_vaccine']  = train_set['opinion_h1n1_risk']+train_set['opinion_h1n1_sick_from_vacc']




test_set['face_mask_wash_hand'] = test_set['behavioral_face_mask']+test_set['behavioral_wash_hands']
test_set['face_mask_touch_face'] = test_set['behavioral_face_mask'] + test_set['behavioral_touch_face']
test_set['wash_hands_touch_face'] = test_set['behavioral_wash_hands']+test_set['behavioral_touch_face']
test_set['opinion_seas_eff_risk'] = test_set['opinion_seas_vacc_effective']+test_set['opinion_seas_risk']
test_set['opinion_h1n1_eff_risk'] = test_set['opinion_h1n1_vacc_effective']+test_set['opinion_h1n1_risk']
test_set['opinio_seas_vacc_eff_sick'] = test_set['opinion_seas_vacc_effective'] + test_set['opinion_seas_sick_from_vacc']
test_set['opinon_h1n1_vacc_eff_sick'] = test_set['opinion_h1n1_vacc_effective']+test_set['opinion_h1n1_sick_from_vacc']
test_set['h1n1_concern_knowledge']   = test_set['h1n1_concern'] + test_set['h1n1_knowledge']
test_set['behaviour_large_gath_outside_home']  = test_set['behavioral_large_gatherings'] + test_set['behavioral_outside_home']
test_set['opinion_seas_vaccine']  = test_set['opinion_seas_risk']+test_set['opinion_seas_sick_from_vacc']
test_set['opinion_h1n1_vaccine']  = test_set['opinion_h1n1_risk']+test_set['opinion_h1n1_sick_from_vacc']



In [ ]:
to_drop = ['behavioral_face_mask','behavioral_wash_hands','behavioral_touch_face','opinion_seas_vacc_effective',
           'opinion_seas_risk','opinion_h1n1_vacc_effective','opinion_h1n1_risk','opinion_seas_vacc_effective',
           'opinion_seas_sick_from_vacc','opinion_h1n1_vacc_effective','opinion_h1n1_sick_from_vacc','h1n1_concern',
           'h1n1_knowledge','behavioral_large_gatherings','behavioral_outside_home','household_adults','household_children']

train_set = train_set.drop(to_drop,axis=1)
test_set = test_set.drop(to_drop,axis=1)

In [ ]:
train_set_A = train_set.copy()
train_set_B = train_set.copy()

In [ ]:
train_set_A['h1n1_vaccine'] = y1
train_set_B['seasonal_vaccine'] = y2

In [ ]:
target_mean_A = train_set_A.groupby(['clusters']).mean()[['h1n1_vaccine']]
for i in list(target_mean_A.columns):
  target_mean_A.rename({i:i+"_mean"},axis=1,inplace=True)
train_set_A = train_set_A.merge(target_mean_A,how='left',on='clusters')
test_set_A = test_set.merge(target_mean_A,how="left",on='clusters')


In [ ]:
target_mean_B = train_set_B.groupby(['clusters']).mean()[['seasonal_vaccine']]
for i in list(target_mean_B.columns):
  target_mean_B.rename({i:i+"_mean"},axis=1,inplace=True)
train_set_B = train_set_B.merge(target_mean_B,how='left',on='clusters')
test_set_B = test_set.merge(target_mean_B,how="left",on='clusters')

In [ ]:
freq_enc_index = np.where(train_set_A.dtypes == object)[0];
freq_enc_cols = [train_set_A.columns[x] for x in freq_enc_index];freq_enc_cols[:5]

['age_group', 'education', 'race', 'sex', 'income_poverty']

In [ ]:
train_set_A = train_set_A.fillna(-999)
test_set_A = test_set_A.fillna(-999)

train_set_B = train_set_B.fillna(-999)
test_set_B = test_set_B.fillna(-999)

In [ ]:
X_a = train_set_A.drop('h1n1_vaccine',axis=1)
y_a = train_set_A['h1n1_vaccine']
test_A = test_set_A.copy()

X_b = train_set_B.drop('seasonal_vaccine',axis=1)
y_b = train_set_B['seasonal_vaccine']
test_B = test_set_B.copy()



In [ ]:
categorical_A = np.where(X_a.dtypes == object)[0];categorical_A

array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21])

In [ ]:
categorical_B = np.where(X_b.dtypes == object)[0];categorical_B

array([ 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21])

In [ ]:
skf = StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
cat = CatBoostClassifier(iterations=50000,random_state=42,reg_lambda=30,eval_metric='AUC',custom_loss='Logloss')
h1n1_scores = []
h1n1_pred= np.zeros(len(test_A))
for fold, (train_index,test_index) in enumerate(skf.split(X_a,y_a)):
    print(f'====================================== FOLD {fold} ======================================================')
    X_train,X_test = X_a.iloc[train_index],X_a.iloc[test_index]
    y_train,y_test = y_a.iloc[train_index],y_a.iloc[test_index]
    cat.fit(X_train,y_train,early_stopping_rounds=500,eval_set=[(X_train,y_train),(X_test,y_test)],use_best_model=True,cat_features=categorical_A)
    h1n1_scores.append(roc_auc_score(y_test,cat.predict_proba(X_test)[:,1]))
    h1n1_pred +=cat.predict_proba(test_A)[:,1]

In [ ]:
pred1 = h1n1_pred/10

In [ ]:
np.mean(h1n1_scores)

0.8173138634839532

In [ ]:
skf = StratifiedKFold(n_splits=10,random_state=42,shuffle=True)
cat = CatBoostClassifier(iterations=50000,random_state=42,reg_lambda=30,eval_metric='AUC',custom_loss='Logloss')
season_scores = []
season_pred= np.zeros(len(test_B))
for fold, (train_index,test_index) in enumerate(skf.split(X_b,y_b)):
    print(f'====================================== FOLD {fold} ======================================================')
    X_train,X_test = X_b.iloc[train_index],X_b.iloc[test_index]
    y_train,y_test = y_b.iloc[train_index],y_b.iloc[test_index]
    cat.fit(X_train,y_train,early_stopping_rounds=500,eval_set=[(X_train,y_train),(X_test,y_test)],use_best_model=True,cat_features=categorical_B)
    season_scores.append(roc_auc_score(y_test,cat.predict_proba(X_test)[:,1]))
    season_pred +=cat.predict_proba(test_B)[:,1]

In [ ]:
pred2 = season_pred/10

In [ ]:
np.mean(season_scores)

0.782538682062792

In [ ]:
submission['h1n1_vaccine'] = pred1
submission['seasonal_vaccine'] = pred2

In [ ]:
submission.describe()

,respondent_id,h1n1_vaccine,seasonal_vaccine
count,26708.000000,26708.000000,26708.000000
mean,40060.500000,0.212845,0.466110
std,7710.079831,0.199013,0.239023
min,26707.000000,0.016773,0.049880
25%,33383.750000,0.060562,0.253921
50%,40060.500000,0.147472,0.446131
75%,46737.250000,0.266449,0.654844
max,53414.000000,0.907058,0.940776


In [ ]:
submission.corr()

,respondent_id,h1n1_vaccine,seasonal_vaccine
respondent_id,1.000000,-0.003967,0.001742
h1n1_vaccine,-0.003967,1.000000,0.555422
seasonal_vaccine,0.001742,0.555422,1.000000


In [ ]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/PredictH1N1/cat_handle.csv',index=False)

In [ ]:
cat_handle = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PredictH1N1/cat_handle.csv')

In [ ]:
catt = pd.read_csv('cat.csv')

In [ ]:
sta1  = (cat_handle['h1n1_vaccine'] *0.85) + (catt['h1n1_vaccine'] *0.15)

In [ ]:
sta2 = (cat_handle['seasonal_vaccine']*0.85) + (catt['seasonal_vaccine'] *0.15)

In [ ]:
submission['h1n1_vaccine'] = sta1
submission['seasonal_vaccine'] = sta2

In [ ]:
submission.to_csv('stack.csv',index=False)